In [235]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('ML_Data/dm_ready.csv')
df = df.drop(['Lower Bound_x', 'Lower Bound_y', 'Low_y', 'Low_x', 'Upper Bound_y', 'Lower Bound_y', 'Lower Bound_x', 'Upper Bound_x', 'Lower Bound_y', 'Upper Bound_y', 'High', 'Low', 'Upper Bound', 'Lower Bound', 'High_x', 'High_y', 'Unnamed: 0'], axis=1)
df.head()


,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),label
0,2019-05-04,21,11,0.343750,0.0,0.0,0.00000,0.0,0.0,0.00000,0
1,2019-05-03,43,20,0.317460,21.0,11.0,0.34375,0.0,0.0,0.00000,1
2,2019-05-02,39,21,0.350000,43.0,20.0,0.31746,21.0,11.0,0.34375,1
3,2019-05-01,18,11,0.379310,39.0,21.0,0.35000,43.0,20.0,0.31746,0
4,2019-04-30,18,8,0.307692,18.0,11.0,0.37931,39.0,21.0,0.35000,0


In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 11 columns):
date               448 non-null object
negatives          448 non-null int64
positives          448 non-null int64
ratio              448 non-null float64
negatives (t-1)    448 non-null float64
positives (t-1)    448 non-null float64
ratio (t-1)        448 non-null float64
negatives (t-2)    448 non-null float64
positives (t-2)    448 non-null float64
ratio (t-2)        448 non-null float64
label              448 non-null int64
dtypes: float64(7), int64(3), object(1)
memory usage: 38.6+ KB


In [237]:
# get test data for Aske
#test_df = df.head(10)
#df = df.tail(-10)
# end getting test data

In [238]:

y = df['label']
X = df.drop(['label','date'], axis=1)
X = X.values

# scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)




## Split data

In [239]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Logistic Regression

In [232]:
'''from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0, solver='lbfgs')
lr.fit(X_train, y_train)
lr.predict(X_test)

score = lr.score(X_test, y_test)
print(score)'''

0.4888888888888889


# Support Vector Machine

In [240]:
from sklearn import svm

svm = svm.SVC(gamma = 8, C = 2)
svm.fit(X_train, y_train)
svm.predict(X_test)

score = svm.score(X_test, y_test)
print(score)

0.5333333333333333


# Naive bayes

In [241]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
nb.predict(X_test)

score = nb.score(X_test, y_test)
print(score)

0.4888888888888889


# Random forrest

In [248]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(max_depth = 5, n_estimators = 11, random_state = 1)
rf.fit(X_train, y_train)
yPredRf = rf.predict(X_test)

score = f1_score(yPredRf, y_test)
print(score)

0.4235294117647059


# Output

In [25]:
new_test_df = pd.DataFrame({'date':test_df['date'].values, 'label':pred})

In [15]:
new_test_df.to_csv('ML_Data/new_test_df.csv')